In [1]:
from datetime import datetime as dt
import git
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import collections
import json
import time
from vehicle_stream_pipeline import utils 

In [2]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")

df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="MoDstops"
)
df_edges = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
)
df_edges.rename(columns={"Start #": "start_id", "Ende #": "end_id"}, inplace=True)

rides_df = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
rides_df = rides_df[(rides_df["state"] == "completed")]
rides_df["scheduled_to"] = pd.to_datetime(rides_df["scheduled_to"])

/var/folders/kw/t27bnrpn52lfl7x3g64tdw1r0000gn/T/ipykernel_8277/666691188.py:11: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  rides_df = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")


In [3]:
start_date = rides_df.scheduled_to.min()
start_date = dt.strftime(start_date, "%Y-%m-%d")
end_date = rides_df.scheduled_to.max()
end_date = dt.strftime(end_date, "%Y-%m-%d")
start_date = dt.strptime(start_date, "%Y-%m-%d")
end_date = dt.strptime(end_date, "%Y-%m-%d")

In [4]:
start_month = start_date.month
start_year = start_date.year
end_month = end_date.month
end_year = end_date.year

In [5]:
print(start_month, start_year, end_month, end_year)

7 2021 5 2022


In [6]:
years = list(range(start_year, end_year + 1))
months = list(range(start_month, end_month + 1))

In [7]:
print(months, years)

[] [2021, 2022]


In [8]:
rides_df_1 = rides_df.copy()
sim_rides = 0
if sim_rides != 0:

        new_rides_all = pd.DataFrame(columns=rides_df_1.columns)
        for year in years:
            for month in months:
                new_rides = utils.generateRideSpecs(
                    rides_df_1,
                    new_rides_all,
                    df_stops,
                    df_edges,
                    sim_rides,
                    month,
                    year,
                )
                new_rides_all = pd.concat([new_rides, new_rides_all])

        new_rides_all["simulated"] = True
        rides_df_1["simulated"] = False

        new_rides_all = pd.concat([rides_df_1, new_rides_all])

else:
    new_rides_all = rides_df_1

In [9]:
new_rides_all

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index
0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483.0,1.0,4.65,4.65,0.00,False,STANDARD,11009,6004,completed,NaN,2021-06-30 23:12:47,2021-07-01 07:30:00,2021-07-01 07:22:00,637.0,2021-07-01 07:29:37,2021-07-01 07:32:37,2021-07-01 07:25:00,2021-07-01 07:33:55,2021-07-01 07:34:07,2021-07-01 07:33:08,2021-07-01 07:44:27,2021-07-01 07:44:39,2021-07-01 07:44:44,2021-11-03 16:00:44,0.0,457.0,31.0,696.0,1153.0,658.0,495.0,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bb916271-0627-4196-8ec1-5324e0e1f71d,f07028da-ca7e-4713-9e45-743c71712e80,3040.0,1.0,3.45,1.55,1.55,False,VRN,10003,6004,completed,NaN,2021-07-01 09:21:39,2021-07-01 09:21:39,2021-07-01 09:21:39,779.0,2021-07-01 09:31:38,2021-07-01 09:34:38,2021-07-01 09:24:39,2021-07-01 09:32:07,2021-07-01 09:36:59,2021-07-01 09:35:40,2021-07-01 09:40:59,2021-07-01 09:45:53,2021-07-01 09:42:27,2021-11-15 11:24:38,0.0,599.0,62.0,407.0,1006.0,365.0,641.0,1.12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,f797b4d6-2fd4-442a-aec3-32c7f34c9b3e,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,6511.0,1.0,5.20,3.10,3.10,False,VRN,1009,5002,completed,NaN,2021-07-01 17:11:11,2021-07-01 17:11:11,2021-07-01 17:11:11,645.0,2021-07-01 17:18:56,2021-07-01 17:21:56,2021-07-01 17:14:11,2021-07-01 17:24:39,2021-07-01 17:23:01,2021-07-01 17:22:06,2021-07-01 17:36:25,2021-07-01 17:34:47,2021-07-01 17:32:41,2021-11-15 11:24:38,0.0,465.0,10.0,635.0,1100.0,781.0,319.0,0.81,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5b9be6cf-1b25-43d7-b077-e0ede60ba49a,f07028da-ca7e-4713-9e45-743c71712e80,3040.0,1.0,3.45,1.55,1.55,False,VRN,6004,10003,completed,NaN,2021-07-01 18:24:13,2021-07-01 18:24:13,2021-07-01 18:24:13,489.0,2021-07-01 18:29:22,2021-07-01 18:32:22,2021-07-01 18:27:13,2021-07-01 18:30:25,2021-07-01 18:34:21,2021-07-01 18:33:23,2021-07-01 18:38:27,2021-07-01 18:42:23,2021-07-01 18:39:50,2021-11-15 11:24:38,0.0,309.0,61.0,387.0,696.0,365.0,331.0,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,90ef7121-ff20-40bb-9489-f9e935519443,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,5974.0,1.0,4.90,2.80,2.80,False,VRN,5002,9010,completed,NaN,2021-07-01 18:35:04,2021-07-01 18:35:04,2021-07-01 18:35:04,1135.0,2021-07-01 18:51:18,2021-07-01 18:53:59,2021-07-01 18:38:04,2021-07-01 18:53:37,2021-07-01 18:55:14,2021-07-01 18:53:59,2021-07-01 19:04:18,2021-07-01 19:05:55,2021-07-01 19:04:05,2021-11-15 11:24:38,-19.0,955.0,0.0,606.0,1561.0,717.0,844.0,0.85,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18945,4ebacda2-8da6-47a4-b2bb-fefb4df667a0,a41793c0-fb54-44c6-b877-4412ef66d9ac,3523.0,2.0,6.89,4.74,4.74,False,VRN,1008,12008,completed,NaN,2022-04-30 17:49:24,2022-04-30 19:00:00,2022-04-30 18:52:00,86355.0,2022-04-30 18:51:03,2022-04-30 18:51:15,2022-04-30 18:55:00,2022-04-30 18:53:10,2022-04-30 19:04:04,2022-04-30 19:03:29,2022-04-30 19:00:27,2022-04-30 19:11:09,2022-04-30 19:11:39,2022-04-30 19:11:56,-168.0,-225.0,734.0,490.0,265.0,423.0,-158.0,1.16,NaN,NaN,NaN,NaN,

In [10]:
rides_df_filterd = new_rides_all[
    (new_rides_all["scheduled_to"] > start_date)
    & (new_rides_all["scheduled_to"] < end_date)
]

In [11]:
drives_without_drones = utils.calculate_drives(
    rides_df_filterd, start_date, end_date
)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/vehicle_stream_pipeline/utils.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["scheduled_to"] = pd.to_datetime(df["scheduled_to"])


In [12]:
drives_without_drones

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1004,5,67.000000,0.001738,67.001738
1,1001,1005,19,17.631579,0.001543,17.633122
2,1001,1007,2,167.500000,0.002882,167.502882
3,1001,1008,3,111.666667,0.003538,111.670204
4,1001,1012,2,167.500000,0.005694,167.505694
...,...,...,...,...,...,...
3313,18005,11003,1,335.000000,0.001921,335.001921
3314,18005,12001,1,335.000000,0.010741,335.010741
3315,18005,12006,1,335.000000,0.010486,335.010486
3316,18005,12008,1,335.000000,0.017095,335.017095
